Version 1.0.1

# Pandas basics 

Hi! In this programming assignment you need to refresh your `pandas` knowledge. You will need to do several [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)s and [`join`]()`s to solve the task. 

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

from grader import Grader

In [2]:
DATA_FOLDER = '../readonly/final_project_data/'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))

The dataset we are going to use is taken from the competition, that serves as the final project for this course. You can find complete data description at the [competition web page](https://www.kaggle.com/c/competitive-data-science-final-project/data). To join the competition use [this link](https://www.kaggle.com/t/1ea93815dca248e99221df42ebde3540).

## Grading

We will create a grader instace below and use it to collect your answers. When function `submit_tag` is called, grader will store your answer *locally*. The answers will *not* be submited to the platform immediately so you can call `submit_tag` function as many times as you need. 

When you are ready to push your answers to the platform you should fill your credentials and run `submit` function in the <a href="#Authorization-&-Submission">last paragraph</a>  of the assignment.

In [3]:
grader = Grader()

# Task

Let's start with a simple task. 

<ol start="0">
  <li><b>Print the shape of the loaded dataframes and use [`df.head`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) function to print several rows. Examine the features you are given.</b></li>
</ol>

In [4]:
# YOUR CODE GOES HERE
transactions['date']= pd.to_datetime(transactions['date'], format='%d.%m.%Y')
print(transactions.head(150))

          date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0   2013-01-02               0       59    22154      999.00           1.0
1   2013-01-03               0       25     2552      899.00           1.0
2   2013-01-05               0       25     2552      899.00          -1.0
3   2013-01-06               0       25     2554     1709.05           1.0
4   2013-01-15               0       25     2555     1099.00           1.0
5   2013-01-10               0       25     2564      349.00           1.0
6   2013-01-02               0       25     2565      549.00           1.0
7   2013-01-04               0       25     2572      239.00           1.0
8   2013-01-11               0       25     2572      299.00           1.0
9   2013-01-03               0       25     2573      299.00           3.0
10  2013-01-03               0       25     2574      399.00           2.0
11  2013-01-05               0       25     2574      399.00           1.0
12  2013-01-07           

Now use your `pandas` skills to get answers for the following questions. 
The first question is:

1. ** What was the maximum total revenue among all the shops in September, 2014?** 


* Hereinafter *revenue* refers to total sales minus value of goods returned.

*Hints:*

* Sometimes items are returned, find such examples in the dataset. 
* It is handy to split `date` field into [`day`, `month`, `year`] components and use `df.year == 14` and `df.month == 9` in order to select target subset of dates.
* You may work with `date` feature as with srings, or you may first convert it to `pd.datetime` type with `pd.to_datetime` function, but do not forget to set correct `format` argument.

In [5]:
# YOUR CODE GOES HERE
transactions['date']= pd.to_datetime(transactions['date'], format='%d.%m.%Y')
transactions['revenue'] = transactions['item_price'] * transactions['item_cnt_day']
transactions_result = transactions.loc[(transactions['date'].dt.year == 2014) & (transactions['date'].dt.month == 9 )]

max_revenue = transactions_result.groupby('shop_id')['revenue'].sum().max()
grader.submit_tag('max_revenue', max_revenue)

Current answer for task max_revenue is: 7982852.2


Great! Let's move on and answer another question:

<ol start="2">
  <li><b>What item category generated the highest revenue in summer 2014?</b></li>
</ol>

* Submit `id` of the category found.
    
* Here we call "summer" the period from June to August.

*Hints:*

* Note, that for an object `x` of type `pd.Series`: `x.argmax()` returns **index** of the maximum element. `pd.Series` can have non-trivial index (not `[1, 2, 3, ... ]`).

In [6]:
# YOUR CODE GOES HERE
transactions['date']= pd.to_datetime(transactions['date'], format='%d.%m.%Y')
transactions['revenue'] = transactions['item_price'] * transactions['item_cnt_day']
transactions_r = transactions.loc[(transactions['date'].dt.year == 2014) & (transactions['date'].dt.month >= 6 ) & (transactions['date'].dt.month <= 8 )]

transaction_join = pd.merge(transactions_r, items, how='left', on='item_id')
transaction_join = transaction_join.groupby('item_category_id')['revenue'].sum().reset_index(name ='total_revenue')

item_id = transaction_join['item_category_id'].iloc[transaction_join['total_revenue'].argmax()]


category_id_with_max_revenue = item_id
grader.submit_tag('category_id_with_max_revenue', category_id_with_max_revenue)

Current answer for task category_id_with_max_revenue is: 20


<ol start="3">
  <li><b>How many items are there, such that their price stays constant (to the best of our knowledge) during the whole period of time?</b></li>
</ol>

* Let's assume, that the items are returned for the same price as they had been sold.

In [19]:
# YOUR CODE GOES HERE
a = transactions.groupby('item_id')['item_price'].nunique()
num_items_constant_price = ((a == 1).sum())
grader.submit_tag('num_items_constant_price', num_items_constant_price)

Current answer for task num_items_constant_price is: 5926


Remember, the data can sometimes be noisy.

<ol start="4">
  <li><b>What was the variance of the number of sold items per day sequence for the shop with `shop_id = 25` in December, 2014?</b></li>
</ol>

* Fill `total_num_items_sold` and `days` arrays, and plot the sequence with the code below.
* Then compute variance. Remember, there can be differences in how you normalize variance (biased or unbiased estimate, see [link](https://math.stackexchange.com/questions/496627/the-difference-between-unbiased-biased-estimator-variance)). Compute ***unbiased*** estimate (use the right value for `ddof` argument in `pd.var` or `np.var`).

In [ ]:
shop_id = 25
transactions_r = transactions.loc[(transactions['date'].dt.year == 2014) & (transactions['date'].dt.month == 12 )  & (transactions['shop_id'] == 25)]
transactions_r = transactions_r.groupby('date')['item_cnt_day'].sum().reset_index(name ='total_count')


total_num_items_sold = transactions_r['total_count']
days = transactions_r['date']

# Plot it
plt.plot(days, total_num_items_sold)
plt.ylabel('Num items')
plt.xlabel('Day')
plt.title("Daily revenue for shop_id = 25")
plt.show()

total_num_items_sold_var = np.var(total_num_items_sold, ddof=1)
grader.submit_tag('total_num_items_sold_var', total_num_items_sold_var)

## Authorization & Submission
To submit assignment to Cousera platform, please, enter your e-mail and token into the variables below. You can generate token on the programming assignment page. *Note:* Token expires 30 minutes after generation.

In [ ]:
STUDENT_EMAIL = 'leewd1994@live.com'
STUDENT_TOKEN = '3g8JKXlhCGsFog14'
grader.status()

In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Well done! :)